# Airlift on STM32

In [1]:
%discover

test-esp32  serial:///dev/cu.usbserial-0160B5B8  
test-esp32  ws://10.39.40.135:8266               


## UART configuration

Sample configuration for an [Adafruit Huzzah 32](https://www.adafruit.com/product/3405) stacked on an [Adafruit STM32](https://www.adafruit.com/product/4382). *Note:* to avoid shorting the two 3.3V regulators on the boards it is suggested to clip the 3.3V pin of one of the boards (or wire them side-by-side).

| stm32           | esp32
|-----------------|----------------
| uart 3          | uart 2
| tx = PC6 = D6   | rx = 32 = D6
| rx = PC7 = D5   | tx = 14 = D5

* **Attention:** no hardware flow control! Use large rx buffers.

## Install Airlift on ESP32 WiFi Coprocessor

In [1]:
%connect test-stm32-cop
%rsync
%softreset

Connected to test-stm32-cop @ serial:///dev/cu.usbserial-0143520E
UPDATE  lib/urpc_server/async_server.py

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



## Install Airlift on STM32

*Note:* The default MicroPython STM32 build includes a (non-functional) `network.py` that shadows a file with the same name in the Airlift library. Either rename `network.py` in the Airlift library, or build MicroPython without built-in `network` support. Optional: remove the (non-functional) `usocket` to avoid confusion.

In [1]:
%connect test-stm32 serial
%rsync
%softreset

Connected to test-stm32 @ serial:///dev/cu.usbmodem208E307542521
UPDATE  lib/urpc/client.py
UPDATE  esp32/lib/urpc_server/async_server.py


Device disconnected

## Verify RPC

In [1]:
%connect test-stm32
%rsync
%softreset

from urpc import *
import sys
sys_ = import_('sys')

print("Platforms:")
print("   host:    {}".format(sys.platform))
print("   server:  {}".format(sys_.get_('platform')))

Connected to test-stm32 @ serial:///dev/cu.usbmodem208E307542522
Directories match

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Platforms:
   host:    pyboard
   server:  esp32


In [1]:
from urpc import *

builtins_ = import_('builtins')
builtins_.exec(
"""
import time
for i in range(10):
    print(i, i**4)
""")

0 0
1 1
2 16
3 81
4 256
5 625
6 1296
7 2401
8 4096
9 6561


## Networking

In [1]:
import network

ip = network.WLAN(network.STA_IF).ifconfig()[0]
print("IP address:", ip)

IP address: 10.39.40.129


### Webserver

In [1]:
import socket
import network

ip = network.WLAN(network.STA_IF).ifconfig()[0]

s = socket.socket()
ai = socket.getaddrinfo("0.0.0.0", 80)
addr = ai[0][-1]

s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind(addr)
s.listen(2)

print("waiting for connection at http://{}".format(ip))
for i in range(6):
    ss, addr = s.accept()
    print("connected from", addr)
    req = ss.recv(1024)
    if i < 4:
        ss.send('Hello from Airlift, serving request # {}!'.format(i+1).encode())
    else:
        ss.send(b'So long! Server shutting down.')
    ss.close()
s.close()

print("DONE")

waiting for connection at http://10.39.40.129
connected from ('10.39.40.114', 65290)
connected from ('10.39.40.114', 65291)
connected from ('10.39.40.114', 65292)
connected from ('10.39.40.114', 65293)
connected from ('10.39.40.114', 65294)
connected from ('10.39.40.114', 65295)
DONE


### Requests

*Example:* Weather data from https://home.openweathermap.org/

In [1]:
from requests import get
from secrets import openweathermap_apiid

cnt = 1
url = "http://api.openweathermap.org/data/2.5/find?lat=38.5&lon=-120&cnt={}&appid={}".format(cnt, openweathermap_apiid)

j = get(url).json()
for k,v in j.get('list')[0].items():
    print("{:8} {}".format(k,v))

coord    {'lat': 38.4646, 'lon': -120.0399}
wind     {'speed': 2.77, 'deg': 201}
rain     {'1h': 0.18}
weather  ({'id': 500, 'description': 'light rain', 'icon': '10n', 'main': 'Rain'},)
dt       1613444928
snow     None
clouds   {'all': 100}
main     {'feels_like': 268.28, 'pressure': 1019, 'temp_min': 269.26, 'humidity': 93, 'temp_max': 275.93, 'temp': 272.44}
name     Bear Valley
sys      {'country': 'US'}
id       5326968


### GC

In [1]:
gc_ = import_('gc')
gc_.collect()
print("cop mem_free: ", gc_.mem_free())

import gc
gc.collect()
print("host mem_free:", gc.mem_free())

cop mem_free:  96992
host mem_free: 87728


### Socket Echo Test

Run [echo server](https://realpython.com/python-sockets/#echo-server) on (some) host (e.g. from a Python3 notebook).

In [1]:
import socket
import os

N = 512
SERVER = "10.39.40.114"
PORT = 65432

resp = bytearray(N)

print("create socket ...")
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    ai = socket.getaddrinfo(SERVER, PORT)
    addr = ai[0][-1]

    print("connect to", addr)
    s.connect(addr)
    
    for i in range(5):
        data = bytes(os.urandom(N))
        s.sendall(data)
        s.readinto(resp)
        assert resp == data, "Mismatch!"
except Exception as e:
    print("*****", e)
finally:
    print("Success!")
    s.close()

create socket ...
connect to ('10.39.40.114', 65432)
Success!
